# English

**required libraries to be installed**

In [1]:
#libraries for syllables
!git clone https://github.com/libindic/syllabalizer.git
import os
os.chdir('syllabalizer')
!python setup.py sdist
%cd ../
!pip install libindic.utils
!pip install './syllabalizer/dist/libindic-syllabifier-0.0.1.dev28.tar.gz'

#for sngrams
!pip install SNgramExtractor



Cloning into 'syllabalizer'...
remote: Enumerating objects: 186, done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (186/186), 41.35 KiB | 384.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.
running sdist
[pbr] Writing ChangeLog
[pbr] Generating ChangeLog
[pbr] ChangeLog complete (0.0s)
[pbr] Generating AUTHORS
[pbr] AUTHORS complete (0.0s)
running egg_info
creating libindic_syllabifier.egg-info
writing libindic_syllabifier.egg-info/PKG-INFO
writing dependency_links to libindic_syllabifier.egg-info/dependency_links.txt
writing namespace_packages to libindic_syllabifier.egg-info/namespace_packages.txt
writing requirements to libindic_syllabifier.egg-info/requires.txt
writing top-level names to libindic_syllabifier.egg-info/top_level.txt
writing pbr to libindic_syllabifier.egg-info/pbr.json
[pbr] Processing SOURCES.txt
writing manifest file 'libindic_syllabifier.egg-info/SOURCES.txt'
package init file 'libindic/__init__.py' not found (or 

In [2]:

conda install rpy2

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

**Import libraries**

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
import re
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier
from libindic.syllabifier import Syllabifier
from nltk import everygrams
import rpy2.ipython

from rpy2.robjects import pandas2ri   

pandas2ri.activate()

%reload_ext rpy2.ipython

import shap, time

from sklearn import model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import numpy as np

import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from SNgramExtractor import SNgramExtractor



**Functions**

In [4]:
instance = Syllabifier()

def generateNgram(words, n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    #assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if gram in key:
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    #gram = sorted(gram.items(), key=lambda (_, count): -count)   lambda x: x[1]
    return ['_'.join(x) for x in list(gram.keys())]


def syl(sent):
    ss=[]
    sent=sent.split(' ')
    for word in sent:
        tmp=[]
        try:
            tmp=instance.syllabify(word)
            tmp=tmp.split('-')
            for t in tmp:
                if '/' not in t:
                    ss.append(t.strip())
                else:
                    t=t.split('/')
                    for i in t:
                        ss.append(i.strip())
        except:
            ss.append(word)
    return ss


def feat(sent):
    words=sent.split(' ')
    syllables=syl(sent)

    ngrams=[]

    for i in range(1,8):
        ngrams.extend(generateNgram(syllables,i))

    return ngrams






lr=LogisticRegression(C=20)
svm_clf=SVC(kernel='linear',probability=True)
rfc=RandomForestClassifier(n_estimators=10000)
estimators=[('lr', lr), ('svm_clf', svm_clf),('rfc', rfc)]
ensemble1 = VotingClassifier(estimators, voting='soft')
ensemble2 = VotingClassifier(estimators, voting='hard')

def train_classifiers(x_train,x_test,y_train,y_test):
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    print('LR')
    print(classification_report(y_test, y_pred,digits=4))
    print('-------------------------------------------------------------')
    svm_clf.fit(x_train, y_train)
    y_pred = svm_clf.predict(x_test)
    print('lsvc')
    print(classification_report(y_test, y_pred,digits=4))
    print('-------------------------------------------------------------')
    rfc.fit(x_train, y_train)
    y_pred = rfc.predict(x_test)
    print('RFC')
    print(classification_report(y_test, y_pred,digits=4))
    print('-------------------------------------------------------------')
    ensemble1.fit(x_train, y_train)
    y_pred = ensemble1.predict(x_test)
    print('soft voting')
    print(classification_report(y_test, y_pred,digits=4))
    print('-------------------------------------------------------------')
    ensemble2.fit(x_train, y_train)
    y_pred = ensemble2.predict(x_test)
    print('hard voting')
    print(classification_report(y_test, y_pred,digits=4))
    
    
def get_sngrams(text,gram):
  
    SNgram_obj=SNgramExtractor(text,meta_tag='original',trigram_flag='yes')#,nlp_model=nlp)# for english nlp_model=None--for spanish nlp model should be set into nlp which in beging we downloaded
    output=SNgram_obj.get_SNgram()

    return output[gram].split(' ')


def get_ngram_features(sent): # The reason I do not use NLTK's everygrams to extract the features quickly is because the behavior of my n-gram extractor is modified to remove redundant n-grams. Also, I need to label word and char n-grams to avoid ambiguity
    sentence_tokens = sent.split(' ')
    
    features = {}

    ngrams=get_sngrams(sent,'SNBigram')#bi-sngrams
    for ngram in ngrams:
      features[ngram] = features.get(f'SNBigram({ngram})', 0) + 1

    ngrams=get_sngrams(sent,'SNTrigram')#bi-sngrams
    for ngram in ngrams:
      features[ngram] = features.get(f'SNTrigram({ngram})', 0) + 1
    
    return [x for x in list(features.keys()) if len(x)>1]#pd.DataFrame(features,index=[0])



def ngram_extractor(sent):
    return [''.join(ng) for ng in everygrams(sent.replace(' ', '_ _'), 3, 6) 
            if ' ' not in ng and '\n' not in ng and ng != ('_',)]



def ShapleyFiltering(classifier,x_train_vec,vectorizer,k):
    # Create the shap explainers
    t0 = time.time()
    explainer = shap.TreeExplainer(classifier)
    shap_values_train = explainer.shap_values(x_train_vec)
    t1 = time.time()
    timeit=t1-t0
    print('time to compute Shapley values (s):', timeit)

    # From sparse matrix to data frame
    txt_dense_df = pd.DataFrame(x_train_vec.todense(), columns=vectorizer.get_feature_names())
    shap_values_train_df = pd.DataFrame(shap_values_train, columns=txt_dense_df.columns)

    predictions = classifier.predict(x_train_vec.tocsc())
    
    shap_sum = np.abs(shap_values_train_df).mean(axis=0)
    importance_df = pd.DataFrame([txt_dense_df.columns.tolist(), shap_sum.tolist()]).T
    importance_df.columns = ['column_name', 'shap_importance (log-odds)']
    importance_df = importance_df.sort_values('shap_importance (log-odds)', ascending=False)
    importance_df['shap_importance (%)'] = importance_df['shap_importance (log-odds)'].apply(lambda x: 100*x/np.sum(importance_df['shap_importance (log-odds)']))
    
    topN = k
    topk = importance_df.iloc[0:topN]["column_name"]

    print('Cumulative Importance', np.sum(importance_df.iloc[0:topN]["shap_importance (%)"]))
    
    shap_values_imp = shap_values_train_df[topk]
    return shap_values_imp


def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_label):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return classifier, metrics.accuracy_score(predictions, valid_label), metrics.classification_report(predictions, valid_label,digits=4)

In [5]:
train_en=pd.read_csv('../input/pan2020fake-news-spreaders-task/train_en.csv')

test_en=pd.read_csv('../input/pan2020fake-news-spreaders-task/test_en.csv')


In [6]:
#only for english---skip for spanish
wanted=list("qwertyuioplkjhgfdsazxcvbnmQWERTYUIOPLKJHGFDSAZXCVBNM")
def clean(text):
  text=text.split(' ')
  te=[]
  for w in text:
    w1=''
    for c in w:
      if c in wanted:
        w1+=c
    if len(w1)>2:
      te.append(w1)
    
  return ' '.join(te).strip().lower()
train_en['cleaned']=[clean(x) for x in train_en.tweets]
test_en['cleaned']=[clean(x) for x in test_en.tweets]

**Vectorization**

In [7]:
sentences=list(train_en.cleaned)

from scipy.sparse import hstack
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3))

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 5))

syl_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer=feat)

sn_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer=get_ngram_features)

cs_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer=ngram_extractor)



syl_vectorizer.fit(sentences)
word_vectorizer.fit(sentences)
char_vectorizer.fit(sentences)
sn_vectorizer.fit(sentences)
cs_vectorizer.fit(sentences)



train_word_features = word_vectorizer.transform(train_en.cleaned)
test_word_features = word_vectorizer.transform(test_en.cleaned)

train_char_features = char_vectorizer.transform(train_en.cleaned)
test_char_features = char_vectorizer.transform(test_en.cleaned)


train_syl_features = syl_vectorizer.transform(train_en.cleaned)
test_syl_features = syl_vectorizer.transform(test_en.cleaned)

train_sn_features = sn_vectorizer.transform(train_en.cleaned)
test_sn_features = sn_vectorizer.transform(test_en.cleaned)

train_cs_features = cs_vectorizer.transform(train_en.cleaned)
test_cs_features = cs_vectorizer.transform(test_en.cleaned)

x_train_char_word = hstack([train_char_features, train_word_features])
x_test_char_word = hstack([test_char_features, test_word_features])

x_train_cs_word = hstack([train_cs_features, train_word_features])
x_test_cs_word = hstack([test_cs_features, test_word_features])

x_train_char_syl = hstack([train_char_features, train_syl_features])
x_test_char_syl = hstack([test_char_features, test_syl_features])

x_train_syl_word = hstack([train_syl_features, train_word_features])
x_test_syl_word = hstack([test_syl_features, test_word_features])

x_train_word_sn = hstack([train_word_features, train_sn_features])
x_test_word_sn = hstack([test_word_features, test_sn_features])


x_train_char_syl_word=hstack([train_char_features, train_syl_features,train_word_features ])
x_test_char_syl_word=hstack([test_char_features, test_syl_features,test_word_features ])



In [8]:
y_train=train_en.label
y_test=test_en.label

# Char ngrams

In [9]:
train_classifiers(train_char_features,test_char_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7419    0.6900    0.7150       100
           1     0.7103    0.7600    0.7343       100

    accuracy                         0.7250       200
   macro avg     0.7261    0.7250    0.7247       200
weighted avg     0.7261    0.7250    0.7247       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7527    0.7000    0.7254       100
           1     0.7196    0.7700    0.7440       100

    accuracy                         0.7350       200
   macro avg     0.7362    0.7350    0.7347       200
weighted avg     0.7362    0.7350    0.7347       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7071    0.7000    0.7035       100
           1     0.7030    0.7100    0.7065       100

    accuracy                         0.7050   

# char sequences

In [10]:
train_classifiers(train_cs_features,test_cs_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7527    0.7000    0.7254       100
           1     0.7196    0.7700    0.7440       100

    accuracy                         0.7350       200
   macro avg     0.7362    0.7350    0.7347       200
weighted avg     0.7362    0.7350    0.7347       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7474    0.7100    0.7282       100
           1     0.7238    0.7600    0.7415       100

    accuracy                         0.7350       200
   macro avg     0.7356    0.7350    0.7348       200
weighted avg     0.7356    0.7350    0.7348       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6970    0.6900    0.6935       100
           1     0.6931    0.7000    0.6965       100

    accuracy                         0.6950   

# Word ngrams

In [11]:
train_classifiers(train_word_features,test_word_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7692    0.7000    0.7330       100
           1     0.7248    0.7900    0.7560       100

    accuracy                         0.7450       200
   macro avg     0.7470    0.7450    0.7445       200
weighted avg     0.7470    0.7450    0.7445       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7717    0.7100    0.7396       100
           1     0.7315    0.7900    0.7596       100

    accuracy                         0.7500       200
   macro avg     0.7516    0.7500    0.7496       200
weighted avg     0.7516    0.7500    0.7496       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7216    0.7000    0.7107       100
           1     0.7087    0.7300    0.7192       100

    accuracy                         0.7150   

# Syllable ngrams

In [12]:
train_classifiers(train_syl_features,test_syl_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7973    0.5900    0.6782       100
           1     0.6746    0.8500    0.7522       100

    accuracy                         0.7200       200
   macro avg     0.7360    0.7200    0.7152       200
weighted avg     0.7360    0.7200    0.7152       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.8108    0.6000    0.6897       100
           1     0.6825    0.8600    0.7611       100

    accuracy                         0.7300       200
   macro avg     0.7467    0.7300    0.7254       200
weighted avg     0.7467    0.7300    0.7254       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.5497    0.9400    0.6937       100
           1     0.7931    0.2300    0.3566       100

    accuracy                         0.5850   

# Syntactic ngrams

In [13]:
train_classifiers(train_sn_features,test_sn_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7553    0.7100    0.7320       100
           1     0.7264    0.7700    0.7476       100

    accuracy                         0.7400       200
   macro avg     0.7409    0.7400    0.7398       200
weighted avg     0.7409    0.7400    0.7398       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7553    0.7100    0.7320       100
           1     0.7264    0.7700    0.7476       100

    accuracy                         0.7400       200
   macro avg     0.7409    0.7400    0.7398       200
weighted avg     0.7409    0.7400    0.7398       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6154    0.8800    0.7243       100
           1     0.7895    0.4500    0.5732       100

    accuracy                         0.6650   

## char_word

In [14]:
train_classifiers(x_train_char_word,x_test_char_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7423    0.7200    0.7310       100
           1     0.7282    0.7500    0.7389       100

    accuracy                         0.7350       200
   macro avg     0.7352    0.7350    0.7349       200
weighted avg     0.7352    0.7350    0.7349       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7527    0.7000    0.7254       100
           1     0.7196    0.7700    0.7440       100

    accuracy                         0.7350       200
   macro avg     0.7362    0.7350    0.7347       200
weighted avg     0.7362    0.7350    0.7347       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6931    0.7000    0.6965       100
           1     0.6970    0.6900    0.6935       100

    accuracy                         0.6950   

## char sequenc and word

In [15]:

train_classifiers(x_train_cs_word,x_test_cs_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7526    0.7300    0.7411       100
           1     0.7379    0.7600    0.7488       100

    accuracy                         0.7450       200
   macro avg     0.7452    0.7450    0.7449       200
weighted avg     0.7452    0.7450    0.7449       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7579    0.7200    0.7385       100
           1     0.7333    0.7700    0.7512       100

    accuracy                         0.7450       200
   macro avg     0.7456    0.7450    0.7448       200
weighted avg     0.7456    0.7450    0.7448       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7010    0.6800    0.6904       100
           1     0.6893    0.7100    0.6995       100

    accuracy                         0.6950   

## _char_syl

In [16]:

train_classifiers(x_train_char_syl,x_test_char_syl,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7500    0.6900    0.7188       100
           1     0.7130    0.7700    0.7404       100

    accuracy                         0.7300       200
   macro avg     0.7315    0.7300    0.7296       200
weighted avg     0.7315    0.7300    0.7296       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7614    0.6700    0.7128       100
           1     0.7054    0.7900    0.7453       100

    accuracy                         0.7300       200
   macro avg     0.7334    0.7300    0.7290       200
weighted avg     0.7334    0.7300    0.7290       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6727    0.7400    0.7048       100
           1     0.7111    0.6400    0.6737       100

    accuracy                         0.6900   

## syl_word

In [17]:

train_classifiers(x_train_syl_word,x_test_syl_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7674    0.6600    0.7097       100
           1     0.7018    0.8000    0.7477       100

    accuracy                         0.7300       200
   macro avg     0.7346    0.7300    0.7287       200
weighted avg     0.7346    0.7300    0.7287       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7647    0.6500    0.7027       100
           1     0.6957    0.8000    0.7442       100

    accuracy                         0.7250       200
   macro avg     0.7302    0.7250    0.7234       200
weighted avg     0.7302    0.7250    0.7234       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.5946    0.8800    0.7097       100
           1     0.7692    0.4000    0.5263       100

    accuracy                         0.6400   

##  word_sn

In [18]:

train_classifiers(x_train_word_sn,x_test_word_sn,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7717    0.7100    0.7396       100
           1     0.7315    0.7900    0.7596       100

    accuracy                         0.7500       200
   macro avg     0.7516    0.7500    0.7496       200
weighted avg     0.7516    0.7500    0.7496       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7717    0.7100    0.7396       100
           1     0.7315    0.7900    0.7596       100

    accuracy                         0.7500       200
   macro avg     0.7516    0.7500    0.7496       200
weighted avg     0.7516    0.7500    0.7496       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7228    0.7300    0.7264       100
           1     0.7273    0.7200    0.7236       100

    accuracy                         0.7250   

## char_syl_word

In [19]:
train_classifiers(x_train_char_syl_word,x_test_char_syl_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7609    0.7000    0.7292       100
           1     0.7222    0.7800    0.7500       100

    accuracy                         0.7400       200
   macro avg     0.7415    0.7400    0.7396       200
weighted avg     0.7415    0.7400    0.7396       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7614    0.6700    0.7128       100
           1     0.7054    0.7900    0.7453       100

    accuracy                         0.7300       200
   macro avg     0.7334    0.7300    0.7290       200
weighted avg     0.7334    0.7300    0.7290       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6759    0.7300    0.7019       100
           1     0.7065    0.6500    0.6771       100

    accuracy                         0.6900   

In [20]:
print('number of char ngrams: ', train_char_features.shape)
print('number of word ngrams: ', train_word_features.shape)
print('number of char sequences: ', train_cs_features.shape)
print('number of syllable ngrams: ', train_syl_features.shape)
print('number of syntactic ngrams: ', train_sn_features.shape)
print('number of char+word ngrams: ', x_train_char_word.shape)
print('number of word + sn ngrams: ', x_train_word_sn.shape)
print('number of char sequences+ words: ', x_train_cs_word.shape)
print('number of char + syllable ngrams: ', x_train_char_syl.shape)
print('number of syllable + word ngrams: ', x_train_syl_word.shape)
print('number of char + syllable + words ngrams: ', x_train_char_syl_word.shape)

number of char ngrams:  (300, 217430)
number of word ngrams:  (300, 501021)
number of char sequences:  (300, 199898)
number of syllable ngrams:  (300, 2508011)
number of syntactic ngrams:  (300, 294884)
number of char+word ngrams:  (300, 718451)
number of word + sn ngrams:  (300, 795905)
number of char sequences+ words:  (300, 700919)
number of char + syllable ngrams:  (300, 2725441)
number of syllable + word ngrams:  (300, 3009032)
number of char + syllable + words ngrams:  (300, 3226462)


# Seleceting top frequent features only

In [21]:
sentences=list(train_en.cleaned)

from scipy.sparse import hstack
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    max_features=30000,
    ngram_range=(1, 3))

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    max_features=30000,
    ngram_range=(2, 5))

syl_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    max_features=30000,
    analyzer=feat)

sn_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    max_features=30000,
    analyzer=get_ngram_features)

cs_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    max_features=30000,
    analyzer=ngram_extractor)



syl_vectorizer.fit(sentences)
word_vectorizer.fit(sentences)
char_vectorizer.fit(sentences)
sn_vectorizer.fit(sentences)
cs_vectorizer.fit(sentences)



train_word_features = word_vectorizer.transform(train_en.cleaned)
test_word_features = word_vectorizer.transform(test_en.cleaned)

train_char_features = char_vectorizer.transform(train_en.cleaned)
test_char_features = char_vectorizer.transform(test_en.cleaned)


train_syl_features = syl_vectorizer.transform(train_en.cleaned)
test_syl_features = syl_vectorizer.transform(test_en.cleaned)

train_sn_features = sn_vectorizer.transform(train_en.cleaned)
test_sn_features = sn_vectorizer.transform(test_en.cleaned)

train_cs_features = cs_vectorizer.transform(train_en.cleaned)
test_cs_features = cs_vectorizer.transform(test_en.cleaned)


x_train_char_word = hstack([train_char_features, train_word_features])
x_test_char_word = hstack([test_char_features, test_word_features])

x_train_cs_word = hstack([train_cs_features, train_word_features])
x_test_cs_word = hstack([test_cs_features, test_word_features])

x_train_char_syl = hstack([train_char_features, train_syl_features])
x_test_char_syl = hstack([test_char_features, test_syl_features])

x_train_syl_word = hstack([train_syl_features, train_word_features])
x_test_syl_word = hstack([test_syl_features, test_word_features])

x_train_word_sn = hstack([train_word_features, train_sn_features])
x_test_word_sn = hstack([test_word_features, test_sn_features])


x_train_char_syl_word=hstack([train_char_features, train_syl_features,train_word_features ])
x_test_char_syl_word=hstack([test_char_features, test_syl_features,test_word_features ])

## char ngrams

In [22]:
train_classifiers(train_char_features,test_char_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7500    0.7200    0.7347       100
           1     0.7308    0.7600    0.7451       100

    accuracy                         0.7400       200
   macro avg     0.7404    0.7400    0.7399       200
weighted avg     0.7404    0.7400    0.7399       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7340    0.6900    0.7113       100
           1     0.7075    0.7500    0.7282       100

    accuracy                         0.7200       200
   macro avg     0.7208    0.7200    0.7197       200
weighted avg     0.7208    0.7200    0.7197       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7204    0.6700    0.6943       100
           1     0.6916    0.7400    0.7150       100

    accuracy                         0.7050   

## char sequences

In [23]:
train_classifiers(train_cs_features,test_cs_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7396    0.7100    0.7245       100
           1     0.7212    0.7500    0.7353       100

    accuracy                         0.7300       200
   macro avg     0.7304    0.7300    0.7299       200
weighted avg     0.7304    0.7300    0.7299       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7292    0.7000    0.7143       100
           1     0.7115    0.7400    0.7255       100

    accuracy                         0.7200       200
   macro avg     0.7204    0.7200    0.7199       200
weighted avg     0.7204    0.7200    0.7199       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7312    0.6800    0.7047       100
           1     0.7009    0.7500    0.7246       100

    accuracy                         0.7150   

## word ngrams

In [24]:
train_classifiers(train_word_features,test_word_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7684    0.7300    0.7487       100
           1     0.7429    0.7800    0.7610       100

    accuracy                         0.7550       200
   macro avg     0.7556    0.7550    0.7548       200
weighted avg     0.7556    0.7550    0.7548       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7604    0.7300    0.7449       100
           1     0.7404    0.7700    0.7549       100

    accuracy                         0.7500       200
   macro avg     0.7504    0.7500    0.7499       200
weighted avg     0.7504    0.7500    0.7499       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7711    0.6400    0.6995       100
           1     0.6923    0.8100    0.7465       100

    accuracy                         0.7250   

## Syllables

In [25]:
train_classifiers(train_syl_features,test_syl_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7660    0.7200    0.7423       100
           1     0.7358    0.7800    0.7573       100

    accuracy                         0.7500       200
   macro avg     0.7509    0.7500    0.7498       200
weighted avg     0.7509    0.7500    0.7498       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7273    0.7200    0.7236       100
           1     0.7228    0.7300    0.7264       100

    accuracy                         0.7250       200
   macro avg     0.7250    0.7250    0.7250       200
weighted avg     0.7250    0.7250    0.7250       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6814    0.7700    0.7230       100
           1     0.7356    0.6400    0.6845       100

    accuracy                         0.7050   

## Syntactic ngrams

In [26]:
train_classifiers(train_sn_features,test_sn_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7263    0.6900    0.7077       100
           1     0.7048    0.7400    0.7220       100

    accuracy                         0.7150       200
   macro avg     0.7155    0.7150    0.7148       200
weighted avg     0.7155    0.7150    0.7148       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7368    0.7000    0.7179       100
           1     0.7143    0.7500    0.7317       100

    accuracy                         0.7250       200
   macro avg     0.7256    0.7250    0.7248       200
weighted avg     0.7256    0.7250    0.7248       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6694    0.8300    0.7411       100
           1     0.7763    0.5900    0.6705       100

    accuracy                         0.7100   

## char word

In [27]:
train_classifiers(x_train_char_word,x_test_char_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7634    0.7100    0.7358       100
           1     0.7290    0.7800    0.7536       100

    accuracy                         0.7450       200
   macro avg     0.7462    0.7450    0.7447       200
weighted avg     0.7462    0.7450    0.7447       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7660    0.7200    0.7423       100
           1     0.7358    0.7800    0.7573       100

    accuracy                         0.7500       200
   macro avg     0.7509    0.7500    0.7498       200
weighted avg     0.7509    0.7500    0.7498       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7234    0.6800    0.7010       100
           1     0.6981    0.7400    0.7184       100

    accuracy                         0.7100   

## char sequence + word

In [28]:
train_classifiers(x_train_cs_word,x_test_cs_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7500    0.7200    0.7347       100
           1     0.7308    0.7600    0.7451       100

    accuracy                         0.7400       200
   macro avg     0.7404    0.7400    0.7399       200
weighted avg     0.7404    0.7400    0.7399       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7579    0.7200    0.7385       100
           1     0.7333    0.7700    0.7512       100

    accuracy                         0.7450       200
   macro avg     0.7456    0.7450    0.7448       200
weighted avg     0.7456    0.7450    0.7448       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7234    0.6800    0.7010       100
           1     0.6981    0.7400    0.7184       100

    accuracy                         0.7100   

## char+syllable

In [29]:
train_classifiers(x_train_char_syl,x_test_char_syl,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7396    0.7100    0.7245       100
           1     0.7212    0.7500    0.7353       100

    accuracy                         0.7300       200
   macro avg     0.7304    0.7300    0.7299       200
weighted avg     0.7304    0.7300    0.7299       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7396    0.7100    0.7245       100
           1     0.7212    0.7500    0.7353       100

    accuracy                         0.7300       200
   macro avg     0.7304    0.7300    0.7299       200
weighted avg     0.7304    0.7300    0.7299       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7300    0.7300    0.7300       100
           1     0.7300    0.7300    0.7300       100

    accuracy                         0.7300   

## Syl+words

In [30]:
train_classifiers(x_train_syl_word,x_test_syl_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7667    0.6900    0.7263       100
           1     0.7182    0.7900    0.7524       100

    accuracy                         0.7400       200
   macro avg     0.7424    0.7400    0.7393       200
weighted avg     0.7424    0.7400    0.7393       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7582    0.6900    0.7225       100
           1     0.7156    0.7800    0.7464       100

    accuracy                         0.7350       200
   macro avg     0.7369    0.7350    0.7345       200
weighted avg     0.7369    0.7350    0.7345       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.6881    0.7500    0.7177       100
           1     0.7253    0.6600    0.6911       100

    accuracy                         0.7050   

## word sn

In [31]:

train_classifiers(x_train_word_sn,x_test_word_sn,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7629    0.7400    0.7513       100
           1     0.7476    0.7700    0.7586       100

    accuracy                         0.7550       200
   macro avg     0.7552    0.7550    0.7549       200
weighted avg     0.7552    0.7550    0.7549       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7708    0.7400    0.7551       100
           1     0.7500    0.7800    0.7647       100

    accuracy                         0.7600       200
   macro avg     0.7604    0.7600    0.7599       200
weighted avg     0.7604    0.7600    0.7599       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7831    0.6500    0.7104       100
           1     0.7009    0.8200    0.7558       100

    accuracy                         0.7350   

## char syllable words

In [32]:
train_classifiers(x_train_char_syl_word,x_test_char_syl_word,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7634    0.7100    0.7358       100
           1     0.7290    0.7800    0.7536       100

    accuracy                         0.7450       200
   macro avg     0.7462    0.7450    0.7447       200
weighted avg     0.7462    0.7450    0.7447       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7660    0.7200    0.7423       100
           1     0.7358    0.7800    0.7573       100

    accuracy                         0.7500       200
   macro avg     0.7509    0.7500    0.7498       200
weighted avg     0.7509    0.7500    0.7498       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7300    0.7300    0.7300       100
           1     0.7300    0.7300    0.7300       100

    accuracy                         0.7300   

In [33]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_label):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return classifier, metrics.accuracy_score(predictions, valid_label), metrics.classification_report(predictions, valid_label,digits=4)

In [34]:
# Extereme Gradient Boosting on Count Vectors
classifier, accuracy, confusion_matrix = train_model(xgboost.XGBClassifier(), train_char_features, y_train, test_char_features, y_test)
print("Xgb, Accuracy: ", accuracy)
print(confusion_matrix)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[19:01:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Xgb, Accuracy:  0.635
              precision    recall  f1-score   support

           0     0.6200    0.6392    0.6294        97
           1     0.6500    0.6311    0.6404       103

    accuracy                         0.6350       200
   macro avg     0.6350    0.6351    0.6349       200
weighted avg     0.6355    0.6350    0.6351       200



In [35]:
vocab=ShapleyFiltering(classifier,train_char_features,char_vectorizer,15000)


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


time to compute Shapley values (s): 0.2420811653137207
Cumulative Importance 100.0


In [36]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 5),
    vocabulary= vocab)
char_vectorizer.fit(sentences)

TfidfVectorizer(analyzer='char', ngram_range=(2, 5), strip_accents='unicode',
                sublinear_tf=True,
                vocabulary=          tru      tag         bs      t ba       rai     rl us       bar  \
0    0.890163 -0.016491 -0.471654 -0.130093  0.142820  0.120922 -0.040013   
1    0.882915  0.538169  0.232390 -0.102851  0.197417  0.137908  0.120971   
2    0.617736  0.763886  0.373447 -0.169638  0.174929  0.162985 -0.172969   
3   -0.649243  0.585357  0.294760 -0.141312 -0.224866  0.1...
296 -0.175041  0.059986  0.071891  ...   0.0  0.0    0.0    0.0   0.0   0.0   
297 -0.098283 -0.225431 -0.309183  ...   0.0  0.0    0.0    0.0   0.0   0.0   
298 -0.130871 -0.212710 -0.104905  ...   0.0  0.0    0.0    0.0   0.0   0.0   
299 -0.143807 -0.305772 -0.199977  ...   0.0  0.0    0.0    0.0   0.0   0.0   

     shan  shame  sham  shake  
0     0.0    0.0   0.0    0.0  
1     0.0    0.0   0.0    0.0  
2     0.0    0.0   0.0    0.0  
3     0.0    0.0   0.0    0.0  
4     0.0    

In [37]:
train_char_features = char_vectorizer.transform(train_en.cleaned)
test_char_features = char_vectorizer.transform(test_en.cleaned)

In [38]:
train_classifiers(train_char_features,test_char_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7340    0.6900    0.7113       100
           1     0.7075    0.7500    0.7282       100

    accuracy                         0.7200       200
   macro avg     0.7208    0.7200    0.7197       200
weighted avg     0.7208    0.7200    0.7197       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7419    0.6900    0.7150       100
           1     0.7103    0.7600    0.7343       100

    accuracy                         0.7250       200
   macro avg     0.7261    0.7250    0.7247       200
weighted avg     0.7261    0.7250    0.7247       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7312    0.6800    0.7047       100
           1     0.7009    0.7500    0.7246       100

    accuracy                         0.7150   

In [39]:
train_classifiers(train_word_features,test_word_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7684    0.7300    0.7487       100
           1     0.7429    0.7800    0.7610       100

    accuracy                         0.7550       200
   macro avg     0.7556    0.7550    0.7548       200
weighted avg     0.7556    0.7550    0.7548       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7604    0.7300    0.7449       100
           1     0.7404    0.7700    0.7549       100

    accuracy                         0.7500       200
   macro avg     0.7504    0.7500    0.7499       200
weighted avg     0.7504    0.7500    0.7499       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.7711    0.6400    0.6995       100
           1     0.6923    0.8100    0.7465       100

    accuracy                         0.7250   

In [40]:
# Extereme Gradient Boosting on Count Vectors
classifier, accuracy, confusion_matrix = train_model(xgboost.XGBClassifier(), train_word_features, y_train, test_word_features, y_test)
print("Xgb, Accuracy: ", accuracy)
print(confusion_matrix)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[19:11:33] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Xgb, Accuracy:  0.68
              precision    recall  f1-score   support

           0     0.6500    0.6915    0.6701        94
           1     0.7100    0.6698    0.6893       106

    accuracy                         0.6800       200
   macro avg     0.6800    0.6807    0.6797       200
weighted avg     0.6818    0.6800    0.6803       200



In [41]:
wordvocab=ShapleyFiltering(classifier,train_word_features,word_vectorizer,15000)


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


time to compute Shapley values (s): 0.2196192741394043
Cumulative Importance 100.0


In [42]:
sentences=list(train_en.cleaned)
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    vocabulary=wordvocab)
word_vectorizer.fit(sentences)
    


TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode', sublinear_tf=True,
                vocabulary=        trump   hashtag  url user    future      from  user url       out  \
0    1.248691  0.005699  0.375242  0.181637  0.178165  0.244054  0.359007   
1    0.943193  1.011413  0.256479  0.238480  0.301806  0.266080 -0.325854   
2    1.253020  0.927811  0.340464  0.311555  0.322027  0.097903 -0.182189   
3   -0.659346  0.144531  0.370806  0.225099  0.209283  0.248072...
298 -0.435547 -0.410297 -0.093573  ...            0.0    0.0    0.0   0.0   
299  0.300205  0.233041 -0.133541  ...            0.0    0.0    0.0   0.0   

     twenty  two dead  two daughters  two cutest fans  two cutest  twitters  
0       0.0       0.0            0.0              0.0         0.0       0.0  
1       0.0       0.0            0.0              0.0         0.0       0.0  
2       0.0       0.0            0.0              0.0         0.0       0.0  
3       0.0       0.0            0.0              0

In [43]:
train_word_features = word_vectorizer.transform(train_en.cleaned)
test_word_features = word_vectorizer.transform(test_en.cleaned)

In [44]:
train_classifiers(train_word_features,test_word_features,y_train,y_test)

LR
              precision    recall  f1-score   support

           0     0.7849    0.7300    0.7565       100
           1     0.7477    0.8000    0.7729       100

    accuracy                         0.7650       200
   macro avg     0.7663    0.7650    0.7647       200
weighted avg     0.7663    0.7650    0.7647       200

-------------------------------------------------------------
lsvc
              precision    recall  f1-score   support

           0     0.7849    0.7300    0.7565       100
           1     0.7477    0.8000    0.7729       100

    accuracy                         0.7650       200
   macro avg     0.7663    0.7650    0.7647       200
weighted avg     0.7663    0.7650    0.7647       200

-------------------------------------------------------------
RFC
              precision    recall  f1-score   support

           0     0.8125    0.6500    0.7222       100
           1     0.7083    0.8500    0.7727       100

    accuracy                         0.7500   

char

In [45]:
from sklearn.metrics import accuracy_score, classification_report

def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_label):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return classifier, accuracy_score(predictions, valid_label), classification_report(predictions, valid_label,digits=4)

In [46]:
# Extereme Gradient Boosting on Count Vectors
import xgboost

classifier, accuracy, confusion_matrix = train_model(xgboost.XGBClassifier(), x_train, y_train, x_test, y_test)
print("Xgb, Accuracy: ", accuracy)
print(confusion_matrix)

NameError: name 'x_train' is not defined

word

In [ ]:
sentences=list(train_en.cleaned)
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    max_features=30000)
word_vectorizer.fit(sentences)
    
x_train = word_vectorizer.transform(train_en.cleaned)
x_test = word_vectorizer.transform(test_en.cleaned)


y_train=train_en.label
y_test=test_en.label

In [ ]:
# Extereme Gradient Boosting on Count Vectors
import xgboost

classifier, accuracy, confusion_matrix = train_model(xgboost.XGBClassifier(), x_train, y_train, x_test, y_test)
print("Xgb, Accuracy: ", accuracy)
print(confusion_matrix)

In [ ]:
wordVocab=ShapleyFiltering(classifier,x_train,word_vectorizer,15000)


In [ ]:
sentences=list(train_en.cleaned)
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    vocabulary=charVocab)
word_vectorizer.fit(sentences)
    
x_train = word_vectorizer.transform(train_en.cleaned)
x_test = word_vectorizer.transform(test_en.cleaned)


y_train=train_en.label
y_test=test_en.label

In [ ]:
train_classifiers(x_train,x_test,y_train,y_test)

In [ ]:
from scipy.sparse import hstack

train_word_features = word_vectorizer.transform(train_en.cleaned)
test_word_features = word_vectorizer.transform(test_en.cleaned)

train_char_features = char_vectorizer.transform(train_en.cleaned)
test_char_features = char_vectorizer.transform(test_en.cleaned)


x_train = hstack([train_char_features, train_word_features])#,train_sn_features])
x_test = hstack([test_char_features, test_word_features])#,test_sn_features])


y_train=train_en.label
y_test=test_en.label

In [ ]:
train_classifiers(x_train,x_test,y_train,y_test)